In [1]:
import sys

sys.path.insert(0, '..')
import os
from process_bigraph import pp, pf
from biosimulator_processes.data_model import *
from biosimulator_processes.biosimulator_builder import BiosimulatorBuilder


model_filepath = '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'

os.path.exists(model_filepath)

CobraProcess registered successfully.
CopasiProcess registered successfully.
TelluriumProcess registered successfully.
DeterministicTimeCourseParameterScan registered successfully.


/Users/alex/anaconda3/envs/biosimulator-processess/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:386: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


True

In [2]:
from biosimulator_processes.processes.copasi_process import test_process

test_process()

Exception: process "CopasiProcess" not found

In [6]:
from biosimulator_processes.data_model import dynamic_process_config as DynamicProcessConfig

In [7]:
x = {'a': '234', 'b': 20}
config = DynamicProcessConfig(config=x, age=29)

config

ProcessConfig(a='234', b=20, age=29)

In [8]:
human_config = {
    'species': 'human',
    'origin': 'USA'}

rose_config = DynamicProcessConfig(
    name='rose',
    config=human_config,
    age=29,
    sibilings=1,
    car='Toyota')


rose_config

RoseProcessConfig(species='human', origin='USA', age=29, sibilings=1, car='Toyota')

In [1]:
from biosimulator_processes.data_model import TimeCourseDataclass, BiomodelID

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess not available. Error: 
PLEASE NOTE: Smoldyn is not correctly installed on your system which prevents you from using the SmoldynProcess. Please refer to the README for further information on installing Smoldyn.
TelluriumProcess registered successfully.
DeterministicTimeCourseParameterScan registered successfully.


/Users/alex/anaconda3/envs/biosimulator-processess/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:386: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [2]:
bio = BiomodelID('BIOSIM')

TypeError: BaseClass.__init__() takes 1 positional argument but 2 were given

In [3]:
from biosimulator_processes import CORE

In [4]:
CORE.process_registry.registry

{'console-emitter': process_bigraph.composite.ConsoleEmitter,
 'ram-emitter': process_bigraph.composite.RAMEmitter,
 'CobraProcess': biosimulator_processes.processes.cobra_process.CobraProcess,
 'CopasiProcess': biosimulator_processes.processes.copasi_process.CopasiProcess,
 'TelluriumProcess': biosimulator_processes.processes.tellurium_process.TelluriumProcess,
 'DeterministicTimeCourseParameterScan': biosimulator_processes.steps.parameter_scan.DeterministicTimeCourseParameterScan}